In [1]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
import scipy.optimize as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import math
from tabulate import tabulate
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean, std

In [2]:
df_raw =pd.read_csv('https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv')
#df_raw.columns

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       7214 non-null   int64  
 1   name                     7214 non-null   object 
 2   first                    7214 non-null   object 
 3   last                     7214 non-null   object 
 4   compas_screening_date    7214 non-null   object 
 5   sex                      7214 non-null   object 
 6   dob                      7214 non-null   object 
 7   age                      7214 non-null   int64  
 8   age_cat                  7214 non-null   object 
 9   race                     7214 non-null   object 
 10  juv_fel_count            7214 non-null   int64  
 11  decile_score             7214 non-null   int64  
 12  juv_misd_count           7214 non-null   int64  
 13  juv_other_count          7214 non-null   int64  
 14  priors_count            

In [4]:
df_raw.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [5]:
#calculate days in jail for each individual
df_raw['day_stayed'] = (pd.to_datetime(df_raw["c_jail_out"]) - pd.to_datetime(df_raw["c_jail_in"])).astype('timedelta64[h]') / 24

In [6]:
df_raw.corrwith(df_raw["two_year_recid"])


/var/folders/2j/xl2qzfzd781b4n_vgf720t2h0000gn/T/ipykernel_92929/1188784964.py:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_raw.corrwith(df_raw["two_year_recid"])


id                         0.016355
age                       -0.190323
juv_fel_count              0.092546
decile_score               0.350703
juv_misd_count             0.109310
juv_other_count            0.108004
priors_count               0.275117
days_b_screening_arrest    0.024481
c_days_from_compas        -0.067530
is_recid                   0.940544
r_days_from_arrest         0.043244
violent_recid                   NaN
is_violent_recid           0.356476
decile_score.1             0.350703
v_decile_score             0.295895
priors_count.1             0.275117
start                      0.080000
end                       -0.770794
event                      0.785957
two_year_recid             1.000000
day_stayed                 0.118377
dtype: float64

In [7]:
#select necessay variables we need
# df=df_raw.filter(items=['id','race','sex','age','age_cat','decile_score','juv_fel_count','juv_misd_count','juv_other_count','priors_count','days_b_screening_arrest','c_days_from_compas',
#         'is_recid','is_violent_recid','v_decile_score','score_text','v_score_text','c_charge_degree', 'day_stayed', 'two_year_recid'])
    

In [11]:
#drop irrelevant rows
#race != african_american or Caucasian
#days_stayed greater than 0
idx = np.where(((df_raw['race']=="African-American") |(df_raw['race']=="Caucasian"))
        & ((df_raw['sex']=="Male") |(df_raw['sex']=="Female"))
        & ((df_raw['day_stayed'] > 0)))

df = df_raw.loc[idx]

In [12]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,day_stayed
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,10.041667
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1,1.083333
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1,6.291667
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,...,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0,2.916667
9,13,bo bradac,bo,bradac,2013-11-04,Male,1994-06-10,21,Less than 25,Caucasian,...,Medium,2013-11-04,2015-01-06,2015-01-07,1,0,428,1,1,0.958333


In [13]:
# dummy variables for race (Caucasian vs. African American) and sex (Male vs. Female)
df['sex'].replace(['Male','Female'],[1,0], inplace = True)
df['race'].replace(['Caucasian','African-American'],[1,0], inplace = True)

In [14]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,day_stayed
1,3,kevon dixon,kevon,dixon,2013-01-27,1,1982-01-22,34,25 - 45,0,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,10.041667
2,4,ed philo,ed,philo,2013-04-14,1,1991-05-14,24,Less than 25,0,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1,1.083333
6,8,edward riddle,edward,riddle,2014-02-19,1,1974-07-23,41,25 - 45,1,...,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1,6.291667
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,0,1976-06-03,39,25 - 45,1,...,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0,2.916667
9,13,bo bradac,bo,bradac,2013-11-04,1,1994-06-10,21,Less than 25,1,...,Medium,2013-11-04,2015-01-06,2015-01-07,1,0,428,1,1,0.958333
